In [ ]:
# default_exp core

# Jupyter Trial

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say Hello to someone"
    return f'Hello {to}!'

In [ ]:
say_hello("Krishna")

'Hello Krishna!'

## Helper functions


In [ ]:
#export
# from pyspark.sql.functions import regexp_extract, concat, lit
# from pyspark.sql.functions import col, to_date, to_timestamp, when, date_format
# from pyspark.sql import functions as F
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
# from pyspark.sql.window import Window
# from pyspark.sql.functions import countDistinct

# Helper functions

# Compute the time in hours between the given columns. 
# Used for computing the time in hours between two timestamps. 
def computeDayHours(col1, col2):
    '''Compute the time in hours between the given columns. 
       Used for computing the time in hours between two timestamps.'''
    return (to_timestamp(col1).cast(LongType()) - to_timestamp(col2).cast(LongType()))/3600



def computeDayHoursRest(col1, col2):
    '''Used to compute the time from first time stamp and in the day given by the second.
       Example: time duration between 5.45 upto midnight. We pass the 5.45 and the day in the col2. 
       timestamp of just day gives us time at 00 hrs, which is the day start. 
       So we compute 24 - [col1_time - day_start]'''
    return 24 - (to_timestamp(col1).cast(LongType()) - to_timestamp(col2).cast(LongType()))/3600



def loadData(spark, dataPath, bootstrap=True):
    '''loads the data into the DF''' 
    csvFile = spark.read.format("csv")\
    .option("header", "true")\
    .option("sep", "\t")\
    .option("mode", "FAILFAST")\
    .option("inferSchema", "true")\
    .load(dataPath)\
    .coalesce(8)\
    .select('dK','yM','sT','eT','hr','hMo','mMB','mb','mo','vcId')
    
    # This conf setting is required to allow the below queries to work correctly. 
    spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
    csvFile = csvFile.withColumn('um', regexp_extract(col('dK'), 'umId_(.*)vcId.*', 1))\
                .withColumn('vm', concat(lit("vm-"), regexp_extract(col('dK'), '.*vm-(.*)', 1)))
    
    return csvFile

def generatePerDayStats(dataFrame):
    ''' This functions takes an interval based dataframe and computes per day statistics '''
    dataFrameByDay = dataFrame.withColumn("startDate", to_date(col("sT"), "yyyy-MM-d"))\
                    .withColumn("endDate", to_date(col("eT"), "yyyy-MM-d"))\
                    .withColumn("curDate", F.explode(F.expr("sequence(startDate, endDate, interval 1 day)")))\
                    .withColumn("yearMonth", date_format(col("curDate"), 'yyyyMM'))\
                    .withColumn("dayDiff", computeDayHours(col('eT'), col('sT')))\
                    .withColumn("firstPartial", computeDayHoursRest(col('sT'), col('curDate')))\
                    .withColumn("endPartial", computeDayHours(col('eT'), col('curDate')))\
                    .withColumn("dayHours", 
                                when(col("startDate") == col("endDate"), col("dayDiff"))
                                .when(col("startDate") == col("curDate"), col("firstPartial"))
                                .when(col("endDate") == col("curDate"), col("endPartial"))
                                .otherwise(24))\
                    .withColumn('mbhrs', F.expr("dayHours * mb"))\
                    .drop('dayDiff','firstPartial','endPartial', 'dK')

    
    dataFrameDaySum = dataFrameByDay.groupBy(col("um"), col("curDate"))\
             .agg(F.collect_set('hMo').alias('numHost'), F.collect_set('vm').alias('numVms'), \
                  F.sum('dayHours').alias('dayHours'), F.sum('mb').alias('mb'), \
                  F.sum('mbhrs').alias('mbhrs'))
    
    w = (Window()
         .partitionBy(col("um"))
         .orderBy(F.col("curDate"))
         .rowsBetween(-7, 0))

    csvFileDaySumRolling = dataFrameDaySum\
             .withColumn('dayHours_avg', F.avg("dayHours").over(w))\
             .withColumn('mbhrs_avg', F.avg("mbhrs").over(w)) \
             .withColumn('mb_avg', F.avg("mb").over(w))\
             .withColumn('numVms_avg', F.avg("numVms").over(w))\
             .withColumn('numHost_avg', F.avg("numHost").over(w))

    return csvFileDaySumRolling


def generateUMStats(dataFrame):
    ''' This functions takes an interval based dataframe and computes UM statistics '''

    csvFileByDay = dataFrame.withColumn("startDate", to_date(col("sT"), "yyyy-MM-d"))\
                        .withColumn("endDate", to_date(col("eT"), "yyyy-MM-d"))\
                        .withColumn("curDate", F.explode(F.expr("sequence(startDate, endDate, interval 1 day)")))\
                        .withColumn("yearMonth", date_format(col("curDate"), 'yyyyMM'))\
                        .withColumn("dayDiff", computeDayHours(col('eT'), col('sT')))\
                        .withColumn("firstPartial", computeDayHoursRest(col('sT'), col('curDate')))\
                        .withColumn("endPartial", computeDayHours(col('eT'), col('curDate')))\
                        .withColumn("dayHours", 
                                    when(col("startDate") == col("endDate"), col("dayDiff"))
                                    .when(col("startDate") == col("curDate"), col("firstPartial"))
                                    .when(col("endDate") == col("curDate"), col("endPartial"))
                                    .otherwise(24))\
                        .withColumn('mbhrs', F.expr("dayHours * mb"))\
                        .drop('dayDiff','firstPartial','endPartial', 'dK')
    
    csvFileDaySum = csvFileByDay.groupBy(col("um"), col("yearMonth"))\
                 .agg(F.collect_set('hMo').alias('hostCount'), F.collect_set('vm').alias('vmCount'), \
                      F.sum('dayHours').alias('duration'), F.sum('mb').alias('mbCount'), \
                      F.approx_count_distinct('sT', 0.05).alias('recCount'),\
                      F.sum('mbhrs').alias('mbhrs'))

    return csvFileDaySum

def writeData(dataFrame, filename):
    df = dataFrame.toPandas()
    df.to_csv(filename, index=False)

#from pyspark.sql import SparkSession
#from pyspark import SparkConf
def getSpark():
    '''Create a spark context'''
    appName = "UM stats handling"
    master = "local"
    spark = SparkSession\
        .builder\
        .appName(appName)\
        .config("spark.executor.instances", "6")\
        .config("spark.executor.cores", "4")\
        .config("spark.kubernetes.executor.request.cpu", "4")\
        .config("spark.kubernetes.executor.limit.cpu", "4")\
        .config("spark.executor.memory", "5g")\
        .config("spark.kubernetes.executor.request.memory", "5G")\
        .config("spark.kubernetes.executor.limit.memory", "5G")\
        .getOrCreate()
    return spark